Table of Contents

# 1  Settings

## 1.1  Load data
## 1.2  각종 설정들
### 1.2.1  Mapbox, DeckGL
### 1.2.2  Plotly
## 1.3  Helper func 정의
# 2  Data EDA
## 2.1  전반적인 일일 이용수준은 어떨까?
### 2.1.1  이용과 운행
#### 2.1.1.1  이용횟수
### 2.1.2  노선과 정류장
#### 2.1.2.1  정류장 분포
#### 2.1.2.2  노선분포
#### 2.1.2.3  노선당 정류장 수 분포
#### 2.1.2.4  정류장당 노선 수 분포
## 2.2  어느 시간대에 버스를 많이 탈까?
### 2.2.1  시간대별 이용량
## 2.3  출퇴근 시간에 어디에서 어떤 버스를 많이탈까?
### 2.3.1  어디 정류장에서 많이 타고 내릴까?
#### 2.3.1.1  이용량 분포
#### 2.3.1.2  출근시간
#### 2.3.1.3  퇴근시간
#### 2.3.1.4  이용 기준 topn 정류장
#### 2.3.1.5  승하차 중심에 따라 정류장을 분류해볼 수 있지 않을까?
### 2.3.2  어떤 노선을 주로 이용할까?
#### 2.3.2.1  이용량 분포
#### 2.3.2.2  topn 노선
#### 2.3.2.3  출근시간

In [1]:
## Settings
### Load data


In [1]:
import os

# for data aggregation.
import numpy as np
import pandas as pd
import geopandas as gpd

# for data visualisation.
import missingno as msno
#import plotly_express as px
import plotly.plotly as py 
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

from deckgljupyter import Layer as deckgl
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#df_usage = pd.read_pickle('data/df_usage_all.pkl')
#df_route_station = pd.read_pickle('data/df_route_station_all.pkl')
#df_route_info = pd.read_pickle('data/df_route_info.pkl')

In [4]:
df_route_station = pd.read_csv('../data/PJT001_stations_table.csv')
df_route_station.columns = ['STATION_ID','시군명','STATION_NM','정류소영문명','정류소번호','중앙차로여부',
                            '관할관청','위치','MAP_LAT','MAP_LON','모바일정류장ID','이비카드정류장ID']

df_route_info = pd.read_csv('../data/PJT001_routestationinfo.csv')
df_route_info.columns = ['seq','ROUTE_ID','ROUTE_NM','SEQ','STATION_NM','STATION_ID','mobile_id']
df_route_info_raw_org=df_route_info.copy()

df_tripchain=pd.read_csv('../data/PJT001_TripChain.csv')

columns_0 =[]
for col in df_route_station.columns: 
    columns_0.append(col.replace('||', ''))
columns=[]
for col in columns_0: 
    columns.append(col.replace("'",''))
df_route_station.columns=columns
df_route_station_org=df_route_station.copy()

df_route_station_org=pd.merge(df_route_info_raw_org,df_route_station_org, how='left')
df_route_station_raw_org=pd.merge(df_route_info_raw_org,df_route_station_org, how='left')

col = ['ROUTE_ID', 'ROUTE_NM', 'ROUTE_TP', '시작정류장', '종료정류장', 'PERM_VOL', 'COM_NM', 'ADMIN_NM']
route_info_start=[]
route_info_end = []
ROUTE_ID=set(df_route_info['ROUTE_ID'])
for r in ROUTE_ID:
    route_info=df_route_info_raw_org[df_route_info_raw_org.iloc[:,1]==r].sort_values(["SEQ"], ascending=[True])
    route_info_start.append(route_info.head(1).iloc[0])
    route_info_end.append(route_info.tail(1).iloc[0])

route_info_start_end=pd.merge(pd.DataFrame(route_info_start)[['seq','ROUTE_ID','STATION_ID','SEQ','STATION_NM']],pd.DataFrame(route_info_end)[['ROUTE_ID','STATION_ID','SEQ','STATION_NM']],on='ROUTE_ID')
route_info_start_end.columns=['SEQ','ROUTE_ID', 'STATION_ID_start', 'SEQ_start','시작정류장', 'STATION_ID_end', 'SEQ_end', '종료정류장']
df_route_info_org=route_info_start_end.copy()


#df_route_info_raw_org.head()
#df_route_info_org.head()

#df_route_station_org.head()

#동탄면에 있는 모든 정류장
route_station = df_route_station_org.loc[df_route_station_org['위치']=='경기도 화성시 동탄면',:]#109개
df_route_station=route_station
#동탄면에 있는 정류장 각각에 지나는 루트 
STARION_ROUTE = []
STATION_ID=set(route_station['STATION_ID'])
for r in STATION_ID:
    route_info=df_route_info[df_route_info['STATION_ID']==r]
    STARION_ROUTE.append([r,set(route_info['ROUTE_ID'])])
STARION_ROUTE=pd.DataFrame(STARION_ROUTE)
STARION_ROUTE.columns=['STATION_ID', 'ROUTE_ID_set']

#동탄면에 있는 정류장을 하나라도 지나는 모든 루트
ROUTE=pd.merge(route_station[['STATION_ID']],df_route_info,how='left') #528개
ROUTE=pd.DataFrame(ROUTE.dropna()) #514개 (14개는 정류장은 있는데 루트가 없음)
df_route_info_raw=ROUTE.drop_duplicates().copy() #중복 행 발견, 제거 후 469개

# 동탄면 안에서 출발 종료 정류장 정보
col = ['ROUTE_ID', 'ROUTE_NM', 'ROUTE_TP', '시작정류장', '종료정류장', 'PERM_VOL', 'COM_NM', 'ADMIN_NM']
route_info_start=[]
route_info_end = []
ROUTE_ID=set(df_route_info_raw['ROUTE_ID'])
for r in ROUTE_ID:
    route_info=df_route_info_raw[df_route_info_raw["ROUTE_ID"]==r].sort_values(["SEQ"], ascending=[True])
    print(route_info)
    route_info_start.append(route_info.head(1).iloc[0])
    route_info_end.append(route_info.head(int(len(route_info)/2)).tail(1).iloc[0])

route_info_start_end=pd.merge(pd.DataFrame(route_info_start)[['ROUTE_NM','seq','ROUTE_ID','STATION_ID','SEQ','STATION_NM']],pd.DataFrame(route_info_end)[['ROUTE_ID','STATION_ID','SEQ','STATION_NM']],on='ROUTE_ID')

route_info_start_end.columns=['ROUTE_NM','SEQ','ROUTE_ID', 'STATION_ID_start', 'SEQ_start','시작정류장', 'STATION_ID_end', 'SEQ_end', '종료정류장']
df_route_info=route_info_start_end.copy()
df_tripchain

      STATION_ID     seq   ROUTE_ID ROUTE_NM  SEQ STATION_NM  mobile_id
1461   233000120  163797  241317002       19    7   센트럴힐즈.금강    36409.0
1465   233003049  163798  241317002       19    8      반도유보라    55412.0
311    233000996  163799  241317002       19    9     반도3.금강    36488.0
2385   233002364  163805  241317002       19   15        신리천    36704.0
2388   233002363  163834  241317002       19   44        신리천    55258.0
1527   233003048  163840  241317002       19   50      반도유보라    55411.0
1531   233000124  163841  241317002       19   51   금강.센트럴힐즈    55400.0
     STATION_ID     seq   ROUTE_ID ROUTE_NM  SEQ       STATION_NM  mobile_id
120   233002421  163859  241317004     19-1   10    예솔초.호반2차.대원2차    55340.0
351   233000980  163861  241317004     19-1   12         모아.반도아파트    36459.0
519   233000960  163862  241317004     19-1   13          경남.반도1차    36452.0
526   233002435  163863  241317004     19-1   14         영천유.금성백조    55293.0
533   233000961  163864  241317004     

,암호화카드번호||','||트랜잭션ID||','||환승횟수||','||교통카드발행사ID||','||총이용객수||','||사용자구분||','||교통수단CD1||','||교통수단CD2||','||교통수단CD3||','||교통수단CD4||',...,'||하차역ID1||','||하차역ID2||','||하차역ID3||','||하차역ID4||','||하차역ID5||','||최초승차역ID||','||최종하차역ID||','||총이용금액||','||수집건수||','||트립체인완료코드
0,900079696430,56,2,9000923,1,1,500.0,500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116708.0,1350,2,;
1,900079697651,5,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4117269.0,1250,1,;
2,900079698254,32,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4107936.0,1550,1,;
3,900079699257,80,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116717.0,1250,1,;
4,900079701419,64,1,9000923,1,1,530.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116848.0,2050,1,;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,900016808011,42,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4130524.0,1250,1,;
1048571,900016808389,72,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4105383.0,1250,1,;
1048572,900016808481,18,2,3101000,1,1,500.0,583.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4119436.0,1350,2,;
1048573,900016684666,18,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4108146.0,1250,1,;


In [5]:
columns_0 =[]
for col in df_tripchain.columns: 
    columns_0.append(col.replace('||', ''))
columns=[]
for col in columns_0: 
    columns.append(col.replace("'",''))

df_tripchain.columns=columns
df_tripchain['최종하차일시'] = pd.to_datetime(df_tripchain["최종하차일시"], errors='coerce')
before = len(df_tripchain)
df_tripchain.dropna(subset=['최종하차일시'], inplace=True)
after = len(df_tripchain)

print("{} -> {} ({:.2f})".format(before, after, (before-after)/before))

1048575 -> 1036746 (0.01)


In [6]:
df_tripchain.loc[0]

암호화카드번호             900079696430
트랜잭션ID                        56
환승횟수                           2
교통카드발행사ID                9000923
총이용객수                          1
사용자구분                          1
교통수단CD1                      500
교통수단CD2                      500
교통수단CD3                      NaN
교통수단CD4                      NaN
교통수단CD5                      NaN
버스노선ID1               4.1002e+07
버스노선ID2               4.1002e+07
버스노선ID3                      NaN
버스노선ID4                      NaN
버스노선ID5                      NaN
차량ID1                1.41772e+08
차량ID2                1.41772e+08
차량ID3                        NaN
차량ID4                        NaN
차량ID5                        NaN
총통행거리                      11170
총탑승시간                         25
총소요시간                         25
승차일시1                2.01807e+13
승차일시2                2.01807e+13
승차일시3                        NaN
승차일시4                        NaN
승차일시5                        NaN
하차일시1                2.01807e+13
하차일시2     

In [7]:
df_tripchain[['암호화카드번호','환승횟수', '사용자구분','버스노선ID1' ,'승차역ID1','최종하차역ID']]

,암호화카드번호,환승횟수,사용자구분,버스노선ID1,승차역ID1,최종하차역ID
0,900079696430,2,1,41002045.0,4116828.0,4116708.0
1,900079697651,1,1,41031040.0,4117280.0,4117269.0
2,900079698254,1,1,41031121.0,4199619.0,4107936.0
3,900079699257,1,1,41031013.0,4108130.0,4116717.0
4,900079701419,1,1,41020001.0,4100122.0,4116848.0
...,...,...,...,...,...,...
1048570,900016808011,1,1,41067012.0,4130419.0,4130524.0
1048571,900016808389,1,1,41061025.0,4108112.0,4105383.0
1048572,900016808481,2,1,41036131.0,4170340.0,4119436.0
1048573,900016684666,1,1,41036113.0,4116734.0,4108146.0


In [8]:
df_tripchain[df_tripchain['버스노선ID1']==41002045][['최초승차일시','승차역ID1']].sort_values(["최초승차일시"], ascending=[True]).head(50)

,최초승차일시,승차역ID1
57173,20180701045617,4110255.0
50968,20180701045800,4100464.0
163403,20180701045802,4100535.0
48528,20180701045802,4100464.0
64558,20180701045805,4100464.0
121276,20180701045809,4100464.0
173357,20180701045832,4108216.0
154032,20180701045850,4100535.0
92110,20180701045926,4100464.0
106477,20180701050008,4116948.0


In [9]:
df_route_station_mapping = pd.read_csv('../data/PJT001_routestationmapping.csv')
df_route_station_mapping[df_route_station_mapping['표준노선ID']==41002045]

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
40,경기시내,경기고속,4100200,234000024,41002045,720-1


In [10]:
df_route_station_mapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
0,경기시내,경원여객M,2805000,216000044,28050900,M6410
1,경기시내,강화운수,4100100,232000028,41001001,2
2,경기시내,강화운수,4100100,232000029,41001013,88
3,경기시내,강화운수,4100100,232000061,41001020,3000
4,경기시내,강화운수,4100100,232000067,41001024,388
...,...,...,...,...,...,...
2122,경기시내,서현운수,4108800,229000060,41088004,330
2123,경기시내,서현운수,4108800,229000063,41088005,850
2124,경기시내,코레일네트웍스,4108900,213000024,41089001,8507
2125,경기시내,신성교통,4109100,229000102,41091900,M7111


In [44]:
df_tripchain.loc[0]

암호화카드번호             900079696430
트랜잭션ID                        56
환승횟수                           2
교통카드발행사ID                9000923
총이용객수                          1
사용자구분                          1
교통수단CD1                      500
교통수단CD2                      500
교통수단CD3                      NaN
교통수단CD4                      NaN
교통수단CD5                      NaN
버스노선ID1               4.1002e+07
버스노선ID2               4.1002e+07
버스노선ID3                      NaN
버스노선ID4                      NaN
버스노선ID5                      NaN
차량ID1                1.41772e+08
차량ID2                1.41772e+08
차량ID3                        NaN
차량ID4                        NaN
차량ID5                        NaN
총통행거리                      11170
총탑승시간                         25
총소요시간                         25
승차일시1                2.01807e+13
승차일시2                2.01807e+13
승차일시3                        NaN
승차일시4                        NaN
승차일시5                        NaN
하차일시1                2.01807e+13
하차일시2     

In [90]:
df_tripchain.sort_values(["환승횟수"], ascending=[False]).head(100)
df_tripchain[df_tripchain['환승횟수']==1]

,암호화카드번호,트랜잭션ID,환승횟수,교통카드발행사ID,총이용객수,사용자구분,교통수단CD1,교통수단CD2,교통수단CD3,교통수단CD4,...,하차역ID1,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차역ID,최종하차역ID,총이용금액,수집건수,트립체인완료코드
345071,900522629760,42,5,9000921,1,1,203.0,105.0,533.0,500.0,...,NaN,NaN,NaN,4108224.0,4108042.0,NaN,4108042.0,2600,5,;
464610,900146045504,32,5,9000931,1,1,115.0,115.0,570.0,500.0,...,NaN,NaN,NaN,4116900.0,4116916.0,NaN,4116916.0,2800,5,;
759051,900194491859,35,5,9000901,1,1,582.0,500.0,500.0,115.0,...,NaN,NaN,NaN,9036129.0,9034515.0,NaN,9034515.0,1550,5,;
305689,900518869155,87,5,9000921,1,1,500.0,500.0,202.0,500.0,...,NaN,NaN,NaN,4108040.0,4170942.0,NaN,4170942.0,1350,5,;
872617,900514159013,1,5,9000923,1,1,500.0,130.0,115.0,201.0,...,NaN,NaN,NaN,313.0,71072.0,NaN,71072.0,2400,5,;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005119,900159293693,37,5,9000921,1,1,511.0,500.0,231.0,500.0,...,NaN,NaN,NaN,4170974.0,4170542.0,NaN,4170542.0,1750,5,;
665952,900509891487,41,5,9000921,1,1,120.0,202.0,500.0,500.0,...,NaN,NaN,NaN,4107943.0,4100117.0,NaN,4100117.0,1650,5,;
973907,900164976227,29,5,9000922,1,1,511.0,582.0,511.0,500.0,...,NaN,NaN,NaN,4103571.0,4178962.0,NaN,4178962.0,1350,5,;
953741,900507484119,13,5,3101000,1,1,582.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4116897.0,4114849.0,NaN,4114849.0,1550,5,;


,암호화카드번호,트랜잭션ID,환승횟수,교통카드발행사ID,총이용객수,사용자구분,교통수단CD1,교통수단CD2,교통수단CD3,교통수단CD4,...,하차역ID1,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차역ID,최종하차역ID,총이용금액,수집건수,트립체인완료코드
1,900079697651,5,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4117269.0,1250,1,;
2,900079698254,32,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4107936.0,1550,1,;
3,900079699257,80,1,9000923,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116717.0,1250,1,;
4,900079701419,64,1,9000923,1,1,530.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116848.0,2050,1,;
5,900079701419,65,1,9000923,1,1,530.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4100121.0,2050,1,;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,900016807708,75,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4150146.0,1250,1,;
1048570,900016808011,42,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4130524.0,1250,1,;
1048571,900016808389,72,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4105383.0,1250,1,;
1048573,900016684666,18,1,3101000,1,1,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4108146.0,1250,1,;


In [ ]:
## 각종 설정들

In [8]:
#Mapbox, DeckGL
access_token = os.getenv('MAPBOX_ACCESS_TOKEN')
view_options = {
    'center': [126.908, 37.132],
    'zoom': 12,
    'bearing': 0,
    'pitch': 60,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}
# Plotly
plotly_kwargs = {
    'margin' : {'l':250},
    'dimensions': (1000, 600)
}

## Helper func 정의

def get_random_color(opacity=1):
    """
        deckgl.Layer 파라미터 중 getColor 에 값을 주는데 쓰입니다.
    """
    rgba = np.random.choice(range(256), size=3).tolist()
    rgba.append(opacity*255)
    return rgba

def draw_pivot(df, pivot, value, kind, title="", xTitle="", yTitle="", topn=-1, asFigure=False, **kwargs):
    pvt = df.groupby(pivot)[value].sum()
    if topn > -1:
        pvt = pvt.sort_values(ascending=False).iloc[:topn]
        pvt.index = pvt.index.astype('category')
        
        if kind == 'barh':
            pvt = pvt.sort_values(ascending=True)
    
    # layout
    fig = pvt.iplot(kind=kind, title=title, xTitle=xTitle, yTitle=yTitle, asFigure=asFigure)
    layout = {
        'showlegend': False
    }
    layout.update(**kwargs)
    fig.plotly_update({'showlegend':False})
    return fig


def draw_subplots(figures, shape, title="", **kwargs):
    titles = [figure['layout']['title']['text'] for figure in figures]
    base_layout = figures[0]['layout']
    base_layout['title']['text'] = title
    base_layout = base_layout.to_plotly_json()
    
    cf.iplot(cf.subplots(figures,
                         subplot_titles=titles,
                         base_layout=base_layout,
                         shape=shape),
            **kwargs)

## Data EDA
데이터에 대한 자세한 내용은 Part1 에서 다뤘으므로, 여기서는 바로 EDA에 들어가도록 하겠습니다.

전반적인 일일 이용수준은 어떨까?
가장 먼저, 화성시 일일 버스 이용에 대한 전반적인 대표값들을 살펴보겠습니다.

이용과 운행
이용횟수
이용은 크게 승차, 환승, 하차로 나뉘어져 있습니다.
하루의 버스 이용횟수는 하차 수로 유추 할 수 있습니다. 승차의 경우 크게 일반 승차와 환승으로 나눠지지만, 결국 모든 이용객은 하차하기 때문입니다.

In [9]:
df_usage['하차'].sum()

NameError: name 'df_usage' is not defined

In [ ]:
df_usage['하차'].sum()
df_usage[['승차', '하차', '환승']].sum().iplot(kind='bar')
df_ratio_u = df_usage[['승차', '환승']].sum().reset_index()
df_ratio_u.rename(columns={'index': '이용', 0:'횟수'}, inplace=True)

df_ratio_u.iplot(kind='pie', labels='이용', values='횟수', title='승차 이용 비율')
df_usage['노선명'].nunique()
df_usage['표준정류장ID'].nunique()
#정류장 분포
df_route_station.head()
# 정류장 정보를 시각화할 형태로 뽑아냅니다.
data = []
for coord in df_route_station[['MAP_LON', 'MAP_LAT']].values.tolist():
    data.append({
        'coordinates': coord
    })
m = deckgl.Map(**view_options)
m.add(deckgl.GridCellLayer(data,
                          cellSize=30,
                          getElevation=100,
                          getPosition='obj => obj.coordinates',
                          getColor=[255,232,0]))
m.show()

# 노선분포
# 이번엔 노선들을 지도에 노선들이 어디에 주로 분포해있나 보겠습니다.
grp = df_route_station.groupby('ROUTE_ID')
data = []

for name, g_df in grp:
    g_df = g_df.sort_values('SEQ')
    data.append({
        'path' : g_df[['MAP_LON', 'MAP_LAT']].values.tolist()
    })
view = view_options.copy()
view['pitch'] = 0
view['zoom'] = 10

m = deckgl.Map(**view)
m.add(deckgl.PathLayer(data,
                       getColor=[255, 233, 0],
                       getWidth=20,
                       getPath='obj => obj.path',
                       opacity=0.01))
m.show()


In [17]:
색이 진한 구간일수록 그 구간을 이용하는 노선이 많다는 뜻입니다.
이전에 언급했던 각 스팟들 위주로 색이 진한 것을 알 수 있습니다.
한편, 각 스팟들로 한 번에 넘어가는 노선(지도 상에서 직선노선)들도 꽤 많이 보입니다.
다만, 구간들이 시내에 있는 것 만큼 색이 진하지 않으므로, 시내보다 해당 구간의 노선이 별로 없음을 알 수 있습니다.

SyntaxError: invalid syntax (<ipython-input-17-51d61fafc415>, line 1)

In [19]:
# 노선당 정류장 수 분포
station_per_route = df_route_station.groupby('ROUTE_ID')['STATION_ID'].size()
station_per_route.shape
station_per_route.iplot(kind='hist', bins=(0, 250, 5),
                        title="노선당 정류장 수 분포",
                        xTitle="정류장 수")
station_per_route.describe()

(36,)

count    36.000000
mean     10.555556
std       5.427151
min       2.000000
25%       7.000000
50%       9.000000
75%      14.000000
max      23.000000
Name: STATION_ID, dtype: float64

In [ ]:
정류장당 노선 수 분포
이번엔 반대로, 정류장당 노선이 몇개가 들어오는지 보겠습니다.

In [20]:
route_per_station = df_route_station.groupby(['STATION_ID','STATION_NM'])['ROUTE_ID'].size()
route_per_station.iplot(kind='hist', title="정류장당 노선 수 분포", xTitle="노선 수")

In [21]:
route_per_station.mean()
route_per_station.median()
route_per_station.std()

5.507246376811594

4.0

5.707639844994647

In [ ]:
굉장히 skewed 합니다. 아무래도 대표값을 뽑으라면 평균보다는 중간값을 사용해야할 듯합니다.
즉, 정류장당 일반적으로 5개의 노선을 갖는다고 볼 수 있습니다.

일단, 각 정류장별 노선수를 지도에 시각화해서 보겠습니다.

In [22]:
pvt = df_route_station.groupby(['STATION_ID', 'STATION_NM', 'MAP_LON', 'MAP_LAT'])[['ROUTE_ID']].size().reset_index()
data = []
for i, row in pvt.iterrows():
    data.append({
        'coord': row[['MAP_LON', 'MAP_LAT']].values.tolist(),
        '노선수': row[0],
        '정류장명': row['STATION_NM']
    })
    
max_value = pvt[0].max()

In [60]:
m = deckgl.Map(**view_options)
m.add(deckgl.GridCellLayer(
    data=data,
    cellSize=50,
    getPosition='x=>x.coord',
    getColor='obj => [255*obj.{}/{}+60, 0, 0]'.format('노선수', max_value),
    getElevation='x=>x.노선수',
    elevationScale=20,
    pickable=True,
    tooltip=["정류장명", "노선수"]
))
m.show()

#이번에는 너무 극단적인 정류장들,
#즉, 노선 수가 굉장히 많은 정류장들(30개 이상)을 잠깐 살펴보겠습니다.

In [24]:
# 노선 개수가 30개 이상인 정류장만 선택
route_per_station_topn = route_per_station[route_per_station>=30].sort_values()
len(route_per_station_topn)

# 정렬해서 시각화
route_per_station_topn.iplot(kind='barh', title="노선 수가 30개 이상인 정류장들", **plotly_kwargs)


1

In [ ]:
이 정류소들은 일종의 노선 허브역할을 하는 듯 합니다.
실제로 이 정류장들은 어디 위치해있는지 노선과 함께 시각화해서 보겠습니다.

먼저 위 정류장들을 지나는 버스노선부터 다 뽑아옵니다.

In [25]:
route_per_station_topn= pd.merge(route_per_station_topn, 
                                 df_route_station[['STATION_ID', 'STATION_NM', 'MAP_LON', 'MAP_LAT']],
                                 on='STATION_ID', how='left').drop_duplicates()

routes_all = {}
for idx, row in route_per_station_topn.iterrows():
    # 먼저 각 정류장을 지나는 노선들을 뽑아옵니다.
    routes = df_route_station[df_route_station['STATION_ID']==row['STATION_ID']]['ROUTE_ID'].unique()
    
    # 각 노선의 지리정보를 따로 담아둡니다.
    for route in routes:
        # 이미 뽑았던 노선인 경우, 제외합니다.
        if route in routes_all: 
            continue
        idx = df_route_station['ROUTE_ID']==route
        coords = df_route_station[idx][['MAP_LON', 'MAP_LAT']].drop_duplicates().values.tolist()
        
        routes_all[route] = coords

In [26]:
len(routes_all)

14

In [30]:
# 시각화 라이브러리에 들어갈 형태로 바꿔줍니다.
path_all = []
for name, coords in routes_all.items():
    path_all.append({
        '노선': name,
        '총 정류장 수': len(coords),
        'path': coords, 
        'color': get_random_color(0.5),
        'opacity': 0
    })
# 시각화 라이브러리에 들어갈 형태로 바꿔줍니다.
station_topn = []
for idx, row in route_per_station_topn.iterrows():
    station_topn.append({
        "coord": row[['MAP_LON', 'MAP_LAT']].values.tolist(),
        "n_path": row['ROUTE_ID'],
        "name": row['STATION_NM']
    })  

m = deckgl.Map(**view_options)
path_layer = deckgl.PathLayer(path_all, 
                             getWidth=30,
                             getColor='obj => obj.color')
station_layer = deckgl.GridCellLayer(station_topn,
                                    cellSize=70,
                                    getPosition='obj => obj.coord',
                                    getElevation='obj => obj.n_path',
                                    elevationScale=30,
                                    getColor=[255,236,0],
                                    pickable=True,
                                    tooltip=["name", "n_path"])
m.add(path_layer)
m.add(station_layer)
m.show()

In [ ]:
어느 시간대에 버스를 많이 탈까?
시간대별 이용량
이용량은 다음과 같이 정의합니다.
이용량 = 승차량 + 하차량 + 환승량

In [ ]:
df_usage['이용'] = df_usage['승차'] + df_usage['하차'] + df_usage['환승']
grp = df_usage.groupby('시간대')

grp['이용'].sum().iplot(kind='bar', 
                      title="시간대별 버스 이용량",
                      xTitle="시",
                      yTitle="이용량")

In [ ]:
제일 많이 타는 시간은 8시와 18시입니다. 즉 출퇴근 시간에 가장 많이 이용합니다.
핫타임인 이 두 시간대를 조금 더 집중적으로 살펴보도록 하겠습니다.

출퇴근 시간에 어디에서 어떤 버스를 많이탈까?
출퇴근 시간에 사람들이 어디에서 어디로 많이 움직일까요?
또 어떤 노선을 더 이용할까요? 이를 먼저 파악해보겠습니다.

출근시간은 넉넉히 7~9시로,
퇴근시간은 17~19시로 가정합니다.

In [ ]:
morning_time = [7,8,9]
dinner_time = [17,18,19]

df_usage = pd.merge(df_usage, df_route_station[['STATION_ID', 'MAP_LON', 'MAP_LAT']], 
                    left_on='표준정류장ID', right_on='STATION_ID', how='left')\
                    .rename({"MAP_LON": "lon", "MAP_LAT": "lat"}, axis=1)
df_morning = df_usage[df_usage['시간대'].isin(morning_time)]
df_dinner = df_usage[df_usage['시간대'].isin(dinner_time)]

In [ ]:
어디 정류장에서 많이 타고 내릴까?
먼저 어디 정류장에서 이용량이 높은지 보겠습니다.

이용량 분포

In [ ]:
figures = [
    draw_pivot(df_morning, '정류장명', '이용', 'hist', "출근시간 정류장 이용량 분포", xTitle='이용량', asFigure=True),
    draw_pivot(df_dinner, '정류장명', '이용', 'hist', "퇴근시간 정류장 이용량 분포", xTitle='이용량', asFigure=True)
]

draw_subplots(figures, (1, 2))

In [ ]:
이용량 분포를 먼저 살펴보면, 왼쪽으로 굉장히 skewed 되어 있습니다.
그리고 소수의 이용량이 높은 값들이 존재합니다.
즉, 몇몇의 정류장들의 이용량이 압도적으로 높은 것을 의미합니다.

지금은 승차, 하차, 환승을 모두 묶은 이용을 봤습니다.
이를 다시 쪼개어 승차, 하차, 환승별로 정류장 이용량 분포가 다른지 보겠습니다.

In [ ]:
station_usage_m = df_morning.groupby('정류장명')[['승차', '하차', '환승']].sum()
fig1 = station_usage_m.iplot(kind='box', title="출근시간 정류장 이용량", asFigure=True)

station_usage_d = df_dinner.groupby('정류장명')[['승차', '하차', '환승']].sum()
fig2 = station_usage_d.iplot(kind='box', title="퇴근시간 정류장 이용량", asFigure=True)
fig2.plotly_update({'showlegend':False})

draw_subplots([fig1, fig2], (1, 2))

출근시간의 경우, 승차나 환승보다 하차가 압도적으로 많습니다.
승차나 환승한 사람들이, 화성시내 어딘가에서 내린다고 생각해보면 상식적으로 보입니다.

퇴근시간도 출근시간과 같은 형태입니다.
다만 환승의 비율이 좀 더 높아졌습니다.

출퇴근 시간의 승차, 하차, 환승지역이 주로 어디인지 실제로 시각화해서 보겠습니다.

출근시간

In [ ]:
df_usage_m = df_morning.groupby(['표준정류장ID', '정류장명', 'lon', 'lat'])[['승차', '하차', '환승', '이용']]\
    .sum()\
    .reset_index()
df_usage_m.shape
df_usage_m.head()
def draw_station_usage(df):
    for target in ['승차', '하차', '환승']:
        # 시각화를 위한 데이터 형태로 만들어 줍니다.
        data = []
        for i, row in df.iterrows():
            data.append({
                "coord" : row[['lon', 'lat']].values.tolist(),
                "정류장명": row['정류장명'],
                "승차": row['승차'],
                "하차": row['하차'],
                "환승": row['환승'],
            })

        # 지도에 시각화
        target_max = df[target].max()
        m = deckgl.Map(**view_options)
        m.add(deckgl.GridCellLayer(data,
                                  cellSize=70,
                                  getPosition='obj => obj.coord',
                                  getElevation='obj => obj.{}'.format(target),
                                  getColor='obj => [255, 232*obj.{}/{}, 0]'.format(target, target_max),
                                  elevationScale=0.3,
                                  pickable=True,
                                  tooltip=["정류장명", target]))
        m.show()
draw_station_usage(df_usage_m)

In [ ]:
위에서부터 차례대로, 승차, 하차, 환승하는 정류장을 시각화한 것입니다.
다음과 같은 사실들을 알 수 있습니다.

전체적인 이용량이 화성시의 동쪽에 매우 높은 것을 알 수 있습니다.
출근시간에 승차지점과 하차지점이 극단적으로 나뉘어져 있습니다.
환승하는 정류장은 더 압도적으로 편항되어 있습니다.
이번에는 각 이용별로, 주요 정류장이 어딘지 살펴보겠습니다.

In [ ]:
topn = 10

colors = ['blue', 'orange', 'green']
figures = []
for i, target in enumerate(["승차", "하차", "환승"]):
    df_target = df_usage_m.groupby("정류장명")[target].sum().sort_values(ascending=False).iloc[:topn]
    df_target = df_target.sort_values()
    figure = df_target.iplot(kind='barh', title="출근시간 {} top{}".format(target, topn), 
                             color=colors[i], asFigure=True)
    figures.append(figure)

draw_subplots(figures, (1, 3))

In [ ]:
예상했던 대로, 주거지역(아파트, 주거동네 등)에 주로 승차하고,
산업/상업지역(병점역 일대, IT단지)에서 주로 하차하는 형태입니다.

환승의 형태는 각 소규모 지역별 중심 스팟이 있으며, 해당 스팟에서 주로 이뤄지는 편입니다.
눈으로 직접 세보면, 대략 화성시 내에 12~13군데 정도의 스팟들이 있습니다.

퇴근시간

In [ ]:
df_usage_d = df_dinner.groupby(['표준정류장ID', '정류장명', 'lon', 'lat'])[['승차', '하차', '환승', '이용']].sum().reset_index()
df_usage_d.shape
df_usage_d.head()
draw_station_usage(df_usage_d)


In [ ]:
출근시간과 반대되는 형태를 보입니다.
역시 승차와 하차가 극단적으로 차이가 나는 정류장들이 많이 보입니다.

In [ ]:
topn = 10

colors = ['blue', 'orange', 'green']
figures = []
for i, target in enumerate(["승차", "하차", "환승"]):
    df_target = df_usage_d.groupby("정류장명")[target].sum().sort_values(ascending=False).iloc[:topn]
    df_target = df_target.sort_values()
    figure = df_target.iplot(kind='barh', title="퇴근시간 {} top{}".format(target, topn), 
                             color=colors[i], asFigure=True)
    figures.append(figure)

draw_subplots(figures, (1, 3))

In [ ]:
이용 기준 topn 정류장
이번에는 이용량 기준으로 상위에 속하는 정류장들을 뽑아낸 후,
해당 정류장들이 어떻게 주로 이용되는지 출퇴근 시간별로 살펴보겠습니다.

In [ ]:
topn = 10
figures = []

for df_target, time in zip([df_usage_m, df_usage_d], ["출근", "퇴근"]):
    df_target = df_target.groupby("정류장명")[['승차', '하차', '환승', '이용']].sum()
    df_target = df_target.sort_values('이용', ascending=False).iloc[:topn]
    df_target = df_target.sort_values('이용')
    fig = df_target[['승차', '하차', '환승']].iplot(kind='barh', 
                                                barmode='stack',
                                                title="{}시간 이용 기준 top{} 정류장".format(time, topn),
                                                asFigure=True)
    if time == "퇴근":
        fig.plotly_update({'showlegend':False})
    figures.append(fig)

draw_subplots(figures, (1, 2))

In [ ]:
먼저, 출퇴근 시간에 이용량이 높은 정류장들은 어느정도 고정적입니다.
또한, 정류장들의 이용 형태를 보면, 승하차 차이가 뚜렷한 정류장이 있는 반면,
승하차 비율이 비슷한 정류장이 있음을 알 수 있습니다.

승하차 중심에 따라 정류장을 분류해볼 수 있지 않을까?
위의 아이디어를 연장하여, 다음과 같은 생각을 해볼 수 있습니다.

출퇴근 시간에, 승하차 또는 환승이 압도적으로 많이 일어나는, 일종의 쏠림현상이 있는 정류장은 화성시에 얼마나 있을까?
또, 이런 이용 형태 중, 어떤 것이 더 비율이 많을까?

따라서, 각각의 정류장 이용형태의 비율을 먼저 살펴보겠습니다.

In [ ]:
# 유의미한 이용비율을 위해, 너무 이용이 작은 정류장은 집계하지 않겠습니다.
threshold = 50

# 출근시간
pvt_m = df_morning.groupby('정류장명')[['승차', '하차', '환승', '이용']].sum()
pvt_m = pvt_m[pvt_m['이용'] > threshold]
pvt_m = pvt_m.div(pvt_m['이용'], axis=0)
pvt_m = pvt_m.drop('이용', axis=1)

# 퇴근시간
pvt_d = df_dinner.groupby('정류장명')[['승차', '하차', '환승', '이용']].sum()
pvt_d = pvt_d[pvt_d['이용'] > threshold]
pvt_d = pvt_d.div(pvt_d['이용'], axis=0)
pvt_d = pvt_d.drop('이용', axis=1)

pvt_d.head()

In [ ]:
fig1 = pvt_m.iplot(kind='hist', title="출근시간 이용형태 비율 분포", asFigure=True, )
fig2 = pvt_d.iplot(kind='hist', title="퇴근시간 이용형태 비율 분포", asFigure=True, )
fig2.plotly_update({'showlegend':False})

draw_subplots([fig1, fig2], shape=(1,2))

In [ ]:
위 분포를보고 다음과 같이 해석할 수 있습니다.

출근시간의 경우, 그래도 승차비율이 높은 정류장이 많습니다.
한편, 승차한 사람은 하차해야 하는데, 하차비율이 높은 정류장은 반면 높지 않습니다.
즉, 많은 정류장에서 승차하는 사람의 비율이 더 많지만, 하차하는 사람들의 비율이 많은 곳은 많지 않습니다.
일터에 가기위해 여러 주거단지에서 출근하는 양상을 생각해보면, 상식적입니다.
퇴근시간의 경우, 분포도 이와 반대고, 해석도 반대입니다.
한편, 환승분포는 그다지 달라지지 않습니다. 즉, 출퇴근시간이 바뀌어도 환승하는 정류장들은 고정되어 있습니다.

In [ ]:
pvt_m.std()
df_rushour = df_usage[df_usage['시간대'].isin([7,8,9,17,18,19])]
pvt = df_rushour.pivot_table(index=['표준정류장ID'], 
                             columns='시간대', 
                             values=['승차', '하차', '환승'], 
                             aggfunc='sum', fill_value=0)

In [ ]:
def draw_diff(times, pvt):

    pvt = (pvt['승차'] - pvt['하차'])[times].sum(axis=1).reset_index()
    pvt = pd.merge(pvt, df_rushour[['표준정류장ID', '정류장명', 'lat', 'lon']], on='표준정류장ID', how='left').drop_duplicates()
    pvt.rename(columns={0:"diff"}, inplace=True)
    
    threshold = 0
    get_in = pvt[pvt['diff'] > threshold]
    get_out = pvt[pvt['diff'] < -threshold]
    get_out['diff'] = abs(get_out['diff'])

    data = {}
    for usage_type, name in zip([get_in, get_out], ['승차', '하차']):
        data[name] = []

        for i, row in usage_type.iterrows():
            data[name].append({
                "coord": row[['lon', 'lat']].values.tolist(),
                "diff": row['diff'],
                "정류장명": row['정류장명']
            })
        
        
    m = deckgl.Map(**view_options)

    max_value = get_in['diff'].max()
    m.add(deckgl.GridCellLayer(data['승차'],
                              cellSize=70,
                              getPosition='obj => obj.coord',
                              getElevation='obj => obj.{}'.format('diff'),
                              getColor='obj => [0, 232*obj.{}/{}, 0]'.format('diff', max_value),
                              elevationScale=0.3,
                              pickable=True,
                              tooltip=["정류장명", "diff"]))

    max_value = get_out['diff'].max()
    m.add(deckgl.GridCellLayer(data['하차'],
                              cellSize=70,
                              getPosition='obj => obj.coord',
                              getElevation='obj => obj.{}'.format('diff'),
                              getColor='obj => [232*obj.{}/{}, 0, 0]'.format('diff', max_value),
                              elevationScale=0.3,
                              pickable=True,
                              tooltip=["정류장명", "diff"]))
    m.show()

In [ ]:
#출근시간
draw_diff([7,8,9], pvt)
#퇴근시간
draw_diff([17, 18, 19], pvt)

In [ ]:
어떤 노선을 주로 이용할까?
이용량 분포

In [ ]:
figures = [
    draw_pivot(df_morning, '노선명', '이용', 'hist', "출근시간 노선 이용량 분포", xTitle="이용량", asFigure=True),
    draw_pivot(df_dinner, '노선명', '이용', 'hist', "퇴근시간 노선 이용량 분포", xTitle="이용량", asFigure=True)
]

draw_subplots(figures, (1, 2))

In [ ]:
# topn 노선
# 시각화 툴인 plotly 내부의 버그문제로, 노선 앞에 문자열을 붙여줘야 합니다.
df_morning['노선명'] = df_morning['노선명'].map('노선 {}'.format)
df_dinner['노선명'] = df_dinner['노선명'].map('노선 {}'.format)

topn = 10
figures = [
    draw_pivot(df_morning, '노선명', '이용', 'barh', "출근시간 노선 이용량 top{}".format(topn), 
               topn=topn, xTitle="이용량", asFigure=True),
    draw_pivot(df_dinner, '노선명', '이용', 'barh', "퇴근시간 노선 이용량 top{}".format(topn), 
               topn=topn, xTitle="이용량", asFigure=True)
]

draw_subplots(figures, (1, 2))

In [ ]:
출퇴근 시간 top10 노선은 거의 같습니다.
일단, 가장 많이 이용되는 708번 노선이 어떻게 사용되는지 보겠습니다.

출근시간

In [ ]:
def draw_route_bar(route_name, df):
    grp_cols = ['노선명', 'lon', 'lat', '정류장명', '정류장순번']
    df_route = df.groupby(grp_cols)[['승차', '하차', '환승', '이용']].sum().reset_index()
    df_route = df_route[df_route['노선명']==route_name]
    df_route = df_route.sort_values('정류장순번')
    df_route[['정류장명', '승차', '하차', '환승']].set_index('정류장명').iplot(kind='bar', title=route_name)

In [ ]:
draw_route_bar('노선 708', df_morning)

In [ ]:
def draw_route_map(route_name, df, buildings=False):
    # 해당 노선의 정보를 가져옵니다.
    
    grp_cols = ['노선명', 'lon', 'lat', '정류장명', '정류장순번']
    df_route = df.groupby(grp_cols)[['승차', '하차', '환승', '이용']].sum().reset_index()
    df_route = df_route[df_route['노선명']==route_name]
    df_route = df_route.sort_values('정류장순번')
    
    # 데이터를 시각화하기 위한 형태로 만듭니다.
    path_data = [{
        'path': [coord for coord in df_route[['lon', 'lat']].values.tolist()],
        'name': '노선 708'
    }]

    
    colors = [[0, 153, 255], [255, 153, 51], [51, 204, 51]]
    station_data = {}
    for i, usage in enumerate(['승차', '하차', '환승']):
        station_data[usage] = []

        for _, row in df_route.iterrows():
            station_data[usage].append({
                'coord': np.add(row[['lon', 'lat']].values, [i*0.0003, 0]).tolist(),
                'value': row[usage],
                '정류장명': row['정류장명'],
                'color': colors[i],
                '정류장순번': row['정류장순번']
            })

    station_meta = []
    for _, row in df_route.iterrows():
        station_meta.append({
            'coord': np.add(row[['lon', 'lat']].values, [0.0003, 0]).tolist(),
            '정류장명': row['정류장명'],
            '정류장순번': row['정류장순번'],
            '승차': row['승차'],
            '하차': row['하차'],
            '환승': row['환승']
        })
        
        
    # 시각화 합니다.
    view_options['center'] = df_route[['lon', 'lat']].mean().tolist()

    m = deckgl.Map(**view_options)

    # 노선 추가
    m.add(deckgl.PathLayer(path_data,
                           getWidth=50, 
                           getColor=[255,223,0]))
    # 노선 위 정류장 추가
    for k, data in station_data.items():
        m.add(deckgl.GridCellLayer(data,
                                   id=k,
                                   cellSize=20,
                                   elevationScale=1,
                                   getPosition='obj => obj.coord',
                                   getElevation='obj => obj.value',
                                   getColor='obj => obj.color'))
    # 각 정류장 메타데이터 추가
    m.add(deckgl.GridCellLayer(station_meta,
                               id='meta',
                               cellSize=20,
                               elevationScale=5,
                               getPosition='obj => obj.coord',
                               getElevation=0,
                               pickable=True,
                               tooltip=["정류장명", "정류장순번", "승차", "하차", "환승"]))

    m.show(buildings=buildings)

In [ ]:
draw_route_map('노선 708', df_morning, buildings=True)


In [ ]:
동쪽의 주거지역을 쭉 지나, 병점 사거리에서 다시 돌아가는 노선입니다.

나머지 노선들도 살펴보겠습니다.

In [ ]:
draw_route_map('노선 701', df_morning, buildings=True)
draw_route_map('노선 34-1', df_morning, buildings=True)
draw_route_map('노선 34', df_morning, buildings=True)
draw_route_map('노선 92-1', df_morning, buildings=True)
